# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [1]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedOutputStream size changed, may indicate binary 


Octave is ready <oct2py.core.Oct2Py object at 0x7f6c42f44100>

















## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [21]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [22]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moirai:latest
input_ar: None
cpu: False


### Show configuration artifact

In [23]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moirai:latest
input_ar: None
cpu: False


## Build W&B artifact

In [24]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03a_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03a_embeddings


In [25]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [26]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [27]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

CommError: Project mi-santamaria/deepvats does not contain artifact: "moirai:latest"

In [9]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb: Downloading large artifact moment-embedding:latest, 1321.41MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.6


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [10]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'toy:v2'

In [11]:
dvats.config.show_attrdict(enc_run.config)

r: 0.71
w: 30
freq: 1s
alias: toy
epochs: 100
mvp_ws: [10, 30]
stride: 1
time_col: None
data_cols: []
mask_sync: False
use_wandb: True
batch_size: 32
csv_config: {}
data_fpath: ~/data/toy.csv
valid_size: 0.2
mask_future: False
wandb_group: None
analysis_mode: online
artifact_name: toy
mask_stateful: True
norm_by_sample: False
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
norm_use_single_batch: False
norm_use_by_single_batch: [False]


### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [12]:
enc_run.config['batch_size']

32

In [13]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'toy:v2'

In [14]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


In [15]:
df.shape

(550, 3)

In [16]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(521, 3, 30)

In [17]:
timer = ut.Time()
timer.start()

1727276588.427751

In [18]:
embs = get_enc_embs_set_stride_set_batch_size(
    X          = enc_input, 
    enc_learn  = enc_learner, 
    stride     = enc_run.config['stride'],
    cpu        = config.cpu, 
    to_numpy   = True,
    batch_size = enc_run.config['batch_size'],
    verbose    = 1
)

--> get_enc_embs_moment
get_enc_embs_moment | Using CUDA
get_enc_embs_moment | Get Outputs
get_enc_embs_moment | Get Embeddings
get_enc_embs_moment -->


In [19]:
timer.end()
timer.show()

[] Start: 1727276588.427751 | End: 1727276589.7187667 | Duration: 1.291015625 seconds


1.291015625

In [20]:
embs.shape

(521, 1024)